In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from pyspark.sql.functions import round
from pyspark.sql import Window
from pyspark.sql.functions import col, when, log
from pyspark.sql.functions import regexp_replace

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/27 16:53:54 WARN Utils: Your hostname, hexiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.178.12 instead (on interface en0)
22/09/27 16:53:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/27 16:53:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/27 16:53:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/27 16:53:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
final = spark.read.parquet('../data/curated/merchant_info.parquet')
final_1 = final

In [3]:
col_list_1 = final.columns

del_col_list = ['Store_type', 'Revenue_levels', 'merchant_abn','postcode']

for i in del_col_list:
    col_list_1.remove(i)

In [4]:
max_num_list = []
min_num_list = []

for i in col_list_1:
    max_num = float(final.agg({i: "max"}).collect()[0][0])
    min_num = float(final.agg({i: "min"}).collect()[0][0])
    max_num_list.append(max_num)
    min_num_list.append(min_num)

min_num_list

[0.1, 0.0, 165.96, 0.06, 5.19, 3.7259495856741, 0.0, 0.06, 0.0, 0.0, 0.0]

In [5]:
for i in range(0,len(min_num_list)):
    final_1 = final_1.withColumn(col_list_1[i], (col(col_list_1[i]) - min_num_list[i])*1000/(max_num_list[i]-min_num_list[i]))

final_1 = final_1.withColumn('avg_prob_fraud_cus', (final_1.avg_prob_fraud_cus *(-1)))\
    .withColumn('num_of_fraud', (final_1.num_of_fraud*(-1)))
    
final_1 = final_1.select('merchant_abn','Store_type', 'Revenue_levels', 'Take_rate', 'count_of_bigorder', 'Avg_amount_monthly', 'Avg_count_monthly', 'Order_avg_value',	'avg_prob_fraud_cus', 'num_of_fraud', 'count_cus_per_mon', 'fix_cus_num', 'avg_income_percentage', 'avg_age_percentage')

In [6]:
from operator import add
from functools import reduce
final_df1 = final_1.withColumn('score',reduce(add, [F.col(x) for x in final_1.columns[3:14]]))

In [7]:
final_df1 = final_df1.sort(final_df1.score.desc())

In [8]:
col_list_2 = final_df1.columns

del_col_list = ['Store_type', 'Revenue_levels', 'merchant_abn','score']

for i in del_col_list:
    col_list_2.remove(i)

In [9]:
final_df2 = final_df1.withColumn('avg_prob_fraud_cus', (final_df1.avg_prob_fraud_cus *(-1)))\
    .withColumn('num_of_fraud', (final_df1.num_of_fraud*(-1)))

In [10]:
sum_num_list = []

for i in col_list_2:
    sum_num = float(final_df2.agg({i: "sum"}).collect()[0][0])
    sum_num_list.append(sum_num)

In [11]:
for i in range(0,len(sum_num_list)):
    final_df2 = final_df2.withColumn(col_list_2[i], (col(col_list_2[i])/sum_num_list[i]))

In [12]:
col_list_2 = final_df2.columns

del_col_list = ['Store_type', 'Revenue_levels', 'merchant_abn', 'score']

for i in del_col_list:
    col_list_2.remove(i)

k = 1/np.log(final_df2.count())

for i in col_list_2:
    final_df2 = final_df2.withColumn(i, when((col(i) != 0), (col(i) * log(col(i)) *(-k))).otherwise(0))


In [13]:
weight_num = []

for i in col_list_2:
    a = 1 - float(final_df2.agg({i: "sum"}).collect()[0][0])
    weight_num.append(a)

entropy = []
total = sum(weight_num)
for i in weight_num:
    a = i/total
    entropy.append(a)

In [14]:
final_df2 = final_1.withColumn('score', final_1.Take_rate * 0.006151820886695801 + final_1.count_of_bigorder * 0.26032576022118487 +\
                            final_1.Avg_amount_monthly * 0.07286776498637058+ final_1.Avg_count_monthly * 0.11264572734132011 +\
                            final_1.Order_avg_value * 0.08240722365270821 + final_1.avg_prob_fraud_cus * 0.0014478393241986918+ final_1.num_of_fraud * 0.07544702789455915+\
                            final_1.count_cus_per_mon * 0.07105607854405234 + final_1.fix_cus_num  * 0.3127293357155681+\
                            final_1.avg_income_percentage * 0.003982393709074386 + final_1.avg_age_percentage * 0.0009390277242678707)

In [22]:
final_df2 = final_df2.sort(final_df2.score.desc())

In [15]:
#从这开始接着改

In [16]:
col_list_3 = final.columns

del_col_list = ['Store_type', 'Revenue_levels', 'merchant_abn','postcode']

for i in del_col_list:
    col_list_3.remove(i)

mean_num_list = []
sd_num_list = []

for i in col_list_3:
    mean_num = float(final.agg({i: "mean"}).collect()[0][0])
    sd_num = float(final.agg({i: "stddev_pop"}).collect()[0][0])
    mean_num_list.append(mean_num)
    sd_num_list.append(sd_num)


In [17]:
final_3 = final

for i in range(0,len(mean_num_list)):
    final_3 = final_3.withColumn(col_list_3[i], (col(col_list_3[i]) - mean_num_list[i])/(sd_num_list[i]))


final_3 = final_3.withColumn('avg_prob_fraud_cus', (final_3.avg_prob_fraud_cus *(-1)))\
    .withColumn('num_of_fraud', (final_3.num_of_fraud*(-1)))
    
final_3 = final_3.select('merchant_abn','Store_type', 'Revenue_levels', 'Take_rate', 'count_of_bigorder', 'Avg_amount_monthly', 'Avg_count_monthly', 'Order_avg_value',	'avg_prob_fraud_cus', 'num_of_fraud', 'count_cus_per_mon', 'fix_cus_num', 'avg_income_percentage', 'avg_age_percentage')

In [18]:
final_df3 = final_3.withColumn('score',reduce(add, [F.col(x) for x in final_1.columns[3:14]]))
final_df3 = final_df3.sort(final_df3.score.desc())

In [19]:
final_df_weight = final_df1.withColumn('weight_score', 0.05*11*final_df1.Take_rate + 0.025*11*final_df1.count_of_bigorder +\
                            0.15*11*final_df1.Avg_amount_monthly + 0.15*11*final_df1.Avg_count_monthly +\
                            0.025*11*final_df1.Order_avg_value + 0.25*11*final_df1.avg_prob_fraud_cus + 0.25*11*final_df1.num_of_fraud +\
                            0.1*11*final_df1.count_cus_per_mon + 0.05*11*final_df1.fix_cus_num +\
                            0.02*11*final_df1.avg_income_percentage + 0.02*11*final_df1.avg_age_percentage)

In [20]:
final_df_weight = final_df_weight.sort(final_df_weight.weight_score.desc())

In [23]:
final_df2

merchant_abn,Store_type,Revenue_levels,Take_rate,count_of_bigorder,Avg_amount_monthly,Avg_count_monthly,Order_avg_value,avg_prob_fraud_cus,num_of_fraud,count_cus_per_mon,fix_cus_num,avg_income_percentage,avg_age_percentage,score
64203420245,tent and awning s...,c,400.0,0.0,815.055972932642,1000.0,0.45805544283030447,-103.00261844375468,-1000.0,1000.0,1000.0,520.2931741035037,671.5831110113019,485.42741240711126
49891706470,tent and awning s...,a,826.0869565217391,0.0,772.880493678027,948.293451281499,0.45805544283030447,-103.00308583547395,-1000.0,999.880397975736,991.051831841972,414.5546579090338,518.2701782093046,475.77895703901345
24852446429,florists supplies...,c,411.5942028985507,0.0,648.6277491354721,966.1320069686604,0.35973546141470886,-103.00004568869595,-1000.0,999.9177736083184,994.4284990714165,403.7962238931629,534.3330702446725,467.20726912318673
86578477987,"watch, clock, and...",a,917.391304347826,0.0,708.2852802422059,908.8227823287583,0.43357183961504836,-103.00085255309655,-1000.0,999.9626243674174,985.3959142326523,327.7999166601939,567.8988503398783,465.123702087357
89726005175,tent and awning s...,a,856.5217391304348,0.0,708.5812293496131,725.7408343065713,0.5685208337148856,-102.99941866865252,-1000.0,999.6262436741742,905.8754009792335,403.7962238931629,534.3330702446725,419.5373610639217
45629217853,"gift, card, novel...",a,997.1014492753624,0.0,522.7214766824259,749.0743885166197,0.3778571834795441,-102.99720668418355,-1000.0,999.8355472166367,924.826945804491,353.96825075035065,597.2427375677006,415.2739294371393
46804135891,"opticians, optica...",c,410.1449275362319,0.0,438.41565998672814,769.421275397371,0.2903331215919354,-103.00587786087313,-1000.0,999.6262436741742,933.6062806010468,520.2931741035037,671.5831110113019,411.2675285616749
43186523025,florists supplies...,b,633.3333333333333,0.0,719.4837743952911,675.2801683080567,0.6296334496143834,-103.03426592884686,-1000.0,999.2524873483484,868.0145196690867,520.2931741035037,671.5831110113019,402.00564969636355
80324045558,"gift, card, novel...",c,198.55072463768113,0.0,448.04966563758114,645.6461058054683,0.3753509878748329,-103.02503940015356,-1000.0,998.9235817816218,840.8745568124261,520.2931741035037,671.5831110113019,367.68222294363756
68216911708,"computers, comput...",c,427.5362318840579,0.0,487.93955382857985,599.5229062958148,0.4574770899984481,-103.01124803083016,-1000.0,998.549825455796,790.0135066689178,327.7999166601939,567.8988503398783,350.01254774673737
